In [1]:
%load_ext autoreload
%autoreload 2

# Paren Checker

In [2]:
from typing import Callable
import torch
from cases.paren_checker import HighLevelParensBalanceChecker, test_HL_parens_balancer_components, BalancedParensDataset

In [3]:
test_HL_parens_balancer_components()

All Balance tests passed!


True

In [4]:
hl_model = HighLevelParensBalanceChecker()
corr = hl_model.get_correspondence()
dataset = BalancedParensDataset(N_samples=5_000, n_ctx=hl_model.get_ll_model_cfg().n_ctx, seed=42)
train_set, test_set = dataset.get_IIT_train_test_set()

making IIT dataset


In [5]:
print(dataset.get_dataset().shape)
print(dataset.get_dataset()[:10]['tokens'])
print(dataset.get_dataset()[:10]['labels'])
for i in range(10):
    tokens, labels, hl_outputs = dataset.get_dataset()[i]['tokens'], dataset.get_dataset()[i]['labels'], hl_model((torch.tensor(dataset.get_dataset()[i]['tokens'])[None,:], None, None))
    nonzero = (torch.tensor(labels) - hl_outputs[0]).nonzero()
    if nonzero.numel() > 0:
        print(tokens, torch.unique(nonzero[:,0]))
        bad_indices = torch.unique(nonzero[:,0]).tolist()
        for idx in bad_indices:
            print(labels[idx], hl_outputs[0,idx])

(5000, 4)
[[3, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 2], [3, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 2], [3, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 2], [3, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 2], [3, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 2], [3, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 2], [3, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 2], [3, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 2], [3, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 2], [3, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 2]]
[[[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0],

In [6]:
from iit.model_pairs.strict_iit_model_pair import StrictIITModelPair
from iit.utils.index import Ix
import torch

n_epochs = 100
training_args = {
    "batch_size": 256,
    "lr": 0.001,
    "num_workers": 0,
    "use_single_loss": False,
    "behavior_weight": 0., #basically doubles the strict weight's job.
    "iit_weight": 1.,
    "strict_weight": 1.0,
    "clip_grad_norm": 1.0,
    "iit_weight_schedule" : lambda s, i: s,
    "strict_weight_schedule" : lambda s, i: s,
    "behavior_weight_schedule" : lambda s, i: s, #0.955*s if 0.955**i > 0.01 else s, #have behavior weight decay over time
    "early_stop" : True,
    "lr_scheduler": torch.optim.lr_scheduler.LinearLR,
    "scheduler_kwargs": dict(start_factor=1, end_factor=0, total_iters=n_epochs),
    "scheduler_val_metric": ["val/accuracy", "val/IIA"], #for ReduceLRonPlateau
    "scheduler_mode": "max", #for ReduceLRonPlateau
}

class LastTokenStrictIITModelPair(StrictIITModelPair):
    def get_label_idxs(self):
        return Ix[[None,-1]]

    # def loss_fn(self) -> Callable[[torch.Tensor, torch.Tensor], torch.Tensor]:
    #     return torch.nn.CrossEntropyLoss()

ll_model = hl_model.get_ll_model()
model_pair = StrictIITModelPair(hl_model=hl_model, ll_model=ll_model, corr=corr, training_args=training_args)

In [7]:
model_pair.train(
    train_set=train_set,
    test_set=test_set,
    optimizer_cls=torch.optim.AdamW,
    epochs=n_epochs,
    optimizer_kwargs=dict(weight_decay=1e-4),
)

training_args={'batch_size': 256, 'lr': 0.001, 'num_workers': 0, 'early_stop': True, 'lr_scheduler': <class 'torch.optim.lr_scheduler.LinearLR'>, 'scheduler_val_metric': ['val/accuracy', 'val/IIA'], 'scheduler_mode': 'max', 'scheduler_kwargs': {'start_factor': 1, 'end_factor': 0, 'total_iters': 100}, 'clip_grad_norm': 1.0, 'seed': 0, 'detach_while_caching': True, 'iit_weight_schedule': <function <lambda> at 0x17fb9d760>, 'strict_weight_schedule': <function <lambda> at 0x17fb9d080>, 'behavior_weight_schedule': <function <lambda> at 0x17fbfe160>, 'atol': 0.05, 'use_single_loss': False, 'iit_weight': 1.0, 'behavior_weight': 0.0, 'strict_weight': 1.0}


Epoch 1: lr: 9.90e-04, iit_weight: 1.00e+00, behavior_weight: 0.00e+00, strict_weight: 1.00e+00, train/iit_loss: 6.26e-01, train/behavior_loss: 0.00e+00, train/strict_loss: 5.28e-01, val/iit_loss: 2.05e-01, val/IIA: 94.15, val/accuracy: 93.91, val/strict_accuracy: 93.91
Epoch 2: lr: 9.80e-04, iit_weight: 1.00e+00, behavior_weight: 0.00e+00, strict_weight: 1.00e+00, train/iit_loss: 2.07e-01, train/behavior_loss: 0.00e+00, train/strict_loss: 1.59e-01, val/iit_loss: 2.25e-01, val/IIA: 92.46, val/accuracy: 96.38, val/strict_accuracy: 96.38
Epoch 3: lr: 9.70e-04, iit_weight: 1.00e+00, behavior_weight: 0.00e+00, strict_weight: 1.00e+00, train/iit_loss: 1.83e-01, train/behavior_loss: 0.00e+00, train/strict_loss: 8.44e-02, val/iit_loss: 1.29e-01, val/IIA: 96.19, val/accuracy: 98.63, val/strict_accuracy: 98.63
Epoch 4: lr: 9.60e-04, iit_weight: 1.00e+00, behavior_weight: 0.00e+00, strict_weight: 1.00e+00, train/iit_loss: 9.64e-02, train/behavior_loss: 0.00e+00, train/strict_loss: 3.97e-02, val/

# Left > Right

In [20]:
from cases.left_greater import HighLevelLeftGreater, test_HL_left_greater_components, LeftGreaterDataset

test_HL_left_greater_components()

All left greater tests passed!


True

In [24]:
hl_model = HighLevelLeftGreater()
corr = hl_model.get_correspondence()
dataset = LeftGreaterDataset(N_samples=1_000, n_ctx=hl_model.get_ll_model_cfg().n_ctx, seed=42)
train_set, test_set = dataset.get_IIT_train_test_set()
print(hl_model.hook_dict)
print(list(corr.keys()))

making IIT dataset
{'input_hook': HookPoint(), 'left_parens_hook': HookPoint(), 'right_parens_hook': HookPoint(), 'mlp0_hook': HookPoint()}
[input_hook, left_parens_hook, right_parens_hook, mlp0_hook]


In [25]:
ll_model = hl_model.get_ll_model()
model_pair = StrictIITModelPair(hl_model=hl_model, ll_model=ll_model, corr=corr, training_args=training_args)
model_pair.train(
    train_set=train_set,
    test_set=test_set,
    optimizer_cls=torch.optim.AdamW,
    epochs=n_epochs,
    optimizer_kwargs=dict(weight_decay=1e-4),
)

training_args={'batch_size': 256, 'lr': 0.001, 'num_workers': 0, 'early_stop': True, 'lr_scheduler': <class 'torch.optim.lr_scheduler.LinearLR'>, 'scheduler_val_metric': ['val/accuracy', 'val/IIA'], 'scheduler_mode': 'max', 'scheduler_kwargs': {'start_factor': 1, 'end_factor': 0, 'total_iters': 100}, 'clip_grad_norm': 1.0, 'seed': 0, 'detach_while_caching': True, 'iit_weight_schedule': <function <lambda> at 0x17fb9d760>, 'strict_weight_schedule': <function <lambda> at 0x17fb9d080>, 'behavior_weight_schedule': <function <lambda> at 0x17fbfe160>, 'atol': 0.05, 'use_single_loss': False, 'iit_weight': 1.0, 'behavior_weight': 0.0, 'strict_weight': 1.0}


Epoch 1: lr: 9.90e-04, iit_weight: 1.00e+00, behavior_weight: 0.00e+00, strict_weight: 1.00e+00, train/iit_loss: 1.63e+00, train/behavior_loss: 0.00e+00, train/strict_loss: 1.57e+00, val/iit_loss: 1.22e+00, val/IIA: 39.32, val/accuracy: 36.75, val/strict_accuracy: 36.75
Epoch 2: lr: 9.80e-04, iit_weight: 1.00e+00, behavior_weight: 0.00e+00, strict_weight: 1.00e+00, train/iit_loss: 1.10e+00, train/behavior_loss: 0.00e+00, train/strict_loss: 1.06e+00, val/iit_loss: 9.43e-01, val/IIA: 48.45, val/accuracy: 52.61, val/strict_accuracy: 52.61
Epoch 3: lr: 9.70e-04, iit_weight: 1.00e+00, behavior_weight: 0.00e+00, strict_weight: 1.00e+00, train/iit_loss: 8.66e-01, train/behavior_loss: 0.00e+00, train/strict_loss: 8.37e-01, val/iit_loss: 7.67e-01, val/IIA: 72.09, val/accuracy: 74.48, val/strict_accuracy: 74.48
Epoch 4: lr: 9.60e-04, iit_weight: 1.00e+00, behavior_weight: 0.00e+00, strict_weight: 1.00e+00, train/iit_loss: 7.07e-01, train/behavior_loss: 0.00e+00, train/strict_loss: 6.90e-01, val/